In [1]:
%matplotlib widget

In [2]:
import flammkuchen as fl
from matplotlib import pyplot as plt
import numpy as np
from scipy import io
import os
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
from pathlib import Path
import pandas as pd
from luminance_analysis import PooledData, traces_stim_from_path, get_meanresp_during_interval

plt.style.use("figures.mplstyle")

In [3]:
fig_fold = Path(r"C:\Users\otprat\Documents\figures\luminance\manuscript_figures\fig6")

if not os.path.isdir(fig_fold):
    os.mkdir(fig_fold)

## Load all data

In [4]:
master_path = Path(r"\\FUNES\Shared\experiments\E0032_luminance\neat_exps")
# master_path = Path(r"J:\_Shared\GC_IO_luminance\data\neat_exps")
# master_path = Path(r"/Users/luigipetrucco/Desktop/data_dictionaries")

In [5]:
from luminance_analysis.utilities import deconv_resamp_norm_trace, reliability, nanzscore
from skimage.filters import threshold_otsu
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree, to_tree, set_link_color_palette
from luminance_analysis.plotting import plot_clusters_dendro, shade_plot, stim_plot, cluster_cols
from luminance_analysis.clustering import cluster_id_search, find_trunc_dendro_clusters

In [6]:
tau_6f = 5
tau_6s = 8
ker_len = 20
normalization = "zscore"


brain_regions_list = ["GC", "IO", "PC"]
tau_list = [tau_6f, tau_6f, tau_6s]
nan_thr_list = [0, 1, 1]
n_cluster_dict = dict(steps=[8, 6, 8], flashes=[4, 5, 4])

protocol_data_dict = dict()

for protocol in ["steps", "flashes"]:
    #load stimulus of GCs and use it as a the reference for time array and stimulus array:
    stim_ref = PooledData(path = master_path / protocol / "GC").stimarray_rep

    # Get number of clusters for each protocol:
    n_cluster_list = n_cluster_dict[protocol]
    
    data_dict = {k:{} for k in brain_regions_list}

    for brain_region, tau, n_cluster, nan_thr in zip(brain_regions_list, tau_list, 
                                                     n_cluster_list, nan_thr_list):
        #Load data :
        path = master_path / protocol / brain_region
        stim, traces, meanresps = traces_stim_from_path(path)

        # Mean traces, calculate reliability index :
        rel_idxs = reliability(traces)

        # Find threshold from reliability histogram...
        rel_thr = threshold_otsu(rel_idxs[~np.isnan(rel_idxs)])

        # ...and load again filtering with the threshold:
        _, traces, meanresps, pooled_data = traces_stim_from_path(path, resp_threshold=rel_thr, nanfraction_thr=nan_thr, return_pooled_data=True)

        # Hierarchical clustering:
        linked = linkage(meanresps, 'ward')

        # Truncate dendrogram at n_cluster level:
        plt.figure(figsize=(0.1, 0.1))  
        dendro = dendrogram(linked, n_cluster, truncate_mode ="lastp")
        plt.close()
        cluster_ids = dendro["leaves"]
        labels = find_trunc_dendro_clusters(linked, dendro) 

        # Deconvolution, resampling / normalization:
        deconv_meanresps = np.empty((meanresps.shape[0], stim_ref.shape[0]))
        resamp_meanresps = np.empty((meanresps.shape[0], stim_ref.shape[0]))
        for roi_i in range(deconv_meanresps.shape[0]):
            deconv_meanresps[roi_i, :] = deconv_resamp_norm_trace(meanresps[roi_i, :], stim[:, 0],
                                                                    stim_ref[:, 0], tau, ker_len,
                                                                    smooth_wnd=4,
                                                                    normalization=normalization)
            resamp_meanresps[roi_i, :] = deconv_resamp_norm_trace(meanresps[roi_i, :], stim[:, 0],
                                                                    stim_ref[:, 0], None, ker_len,
                                                                    smooth_wnd=4,
                                                                    normalization=normalization)

        cluster_resps = np.empty((n_cluster, stim_ref.shape[0]))
        for clust_i in range(n_cluster):
            cluster_resp = np.nanmean(deconv_meanresps[labels==clust_i, :], 0)  # average cluster responses
            cluster_resps[clust_i, :] = nanzscore(cluster_resp)  # normalize

        # Get mean responses of all ROIs to the different luminances of the two step series (upward and downward)
        start_after_stim = 2.5
        post_int_s = 5
        start_after_isi = 4.5
        post_isi_s = 7
        ##Calculate average activity during a luminance value when it was approached from below:
        up_trans_t_sec = np.array([34, 46, 58]) #Times [s.] of upward transitions
        resp_upward = get_meanresp_during_interval(path, up_trans_t_sec, start_after_stim, post_int_s, resp_threshold=rel_thr, nanfraction_thr=nan_thr)
        resp_upward_isi = get_meanresp_during_interval(path, up_trans_t_sec-7, start_after_isi, post_isi_s, resp_threshold=rel_thr, nanfraction_thr=nan_thr)
        ##Calculate average activity during a luminance value when it was approached from above:
        down_trans_t_sec = np.array([101, 89, 77]) #Times [s.] of downward transitions (sorted to match order of upward transitions
        resp_downward = get_meanresp_during_interval(path, down_trans_t_sec, start_after_stim, post_int_s, resp_threshold=rel_thr, nanfraction_thr=nan_thr)
        resp_downward_isi = get_meanresp_during_interval(path, down_trans_t_sec-7, start_after_isi, post_isi_s, resp_threshold=rel_thr, nanfraction_thr=nan_thr)

        # Add everything to dictionary:
        data_dict[brain_region]["linkage_mat"] = linked
        data_dict[brain_region]["clust_labels"] = labels
        data_dict[brain_region]["pooled_data"] = pooled_data
        data_dict[brain_region]["raw_mn_resps"] = meanresps
        data_dict[brain_region]["deconv_mn_resps"] = deconv_meanresps
        data_dict[brain_region]["resamp_mn_resps"] = resamp_meanresps
        data_dict[brain_region]["rel_idxs"] = rel_idxs
        data_dict[brain_region]["rel_thr"] = rel_thr
        data_dict[brain_region]["clust_resps"] = cluster_resps
        
    protocol_data_dict[protocol] = data_dict

[<luminance_analysis.FishData object at 0x0000022120AFC408>, <luminance_analysis.FishData object at 0x0000022120AFC488>, <luminance_analysis.FishData object at 0x0000022120ADDF08>, <luminance_analysis.FishData object at 0x0000022120AD9708>, <luminance_analysis.FishData object at 0x0000022120ADFEC8>]
[<luminance_analysis.FishData object at 0x0000022120AD5EC8>, <luminance_analysis.FishData object at 0x0000022120AD5E48>, <luminance_analysis.FishData object at 0x0000022120AD9DC8>, <luminance_analysis.FishData object at 0x0000022120ADF988>, <luminance_analysis.FishData object at 0x0000022120AE5508>]


C:\tools\miniconda3\envs\rplab\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\tools\miniconda3\envs\rplab\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


[<luminance_analysis.FishData object at 0x00000221217CC348>, <luminance_analysis.FishData object at 0x00000221217CC3C8>, <luminance_analysis.FishData object at 0x00000221217CF988>, <luminance_analysis.FishData object at 0x00000221217D0F48>, <luminance_analysis.FishData object at 0x00000221217D2548>]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[<luminance_analysis.FishData object at 0x00000221219A2B08>, <luminance_analysis.FishData object at 0x00000221219A2B88>, <luminance_analysis.FishData object at 0x00000221219AC108>, <luminance_analysis.FishData object at 0x00000221219AE648>, <luminance_analysis.FishData object at 0x00000221219B6B88>]
[<luminance_analysis.FishData object at 0x00000221219A5BC8>, <luminance_analysis.FishData object at 0x00000221219A5B48>, <luminance_analysis.FishData object at 0x00000221219ACB48>, <luminance_analysis.FishData object at 0x00000221219B4508>, <luminance_analysis.FishData object at 0x0000022121833708>]
[<luminance_analysis.FishData object at 0x000002212182BF88>, <luminance_analysis.FishData object at 0x000002212182BBC8>, <luminance_analysis.FishData object at 0x00000221219ACF48>, <luminance_analysis.FishData object at 0x00000221219B6448>, <luminance_analysis.FishData object at 0x0000022121833C48>]
[<luminance_analysis.FishData object at 0x000002212182BAC8>, <luminance_analysis.FishData object 

...

[<luminance_analysis.FishData object at 0x000002212139DD48>, <luminance_analysis.FishData object at 0x000002212139DCC8>, <luminance_analysis.FishData object at 0x0000022121398808>, <luminance_analysis.FishData object at 0x000002212139C808>, <luminance_analysis.FishData object at 0x0000022121396288>]
[<luminance_analysis.FishData object at 0x000002212139D288>, <luminance_analysis.FishData object at 0x000002212139D4C8>, <luminance_analysis.FishData object at 0x0000022121398148>, <luminance_analysis.FishData object at 0x000002212139C708>, <luminance_analysis.FishData object at 0x00000221216E5908>]
[<luminance_analysis.FishData object at 0x000002212138D3C8>, <luminance_analysis.FishData object at 0x0000022120F15908>, <luminance_analysis.FishData object at 0x0000022120EEA448>, <luminance_analysis.FishData object at 0x00000221216F3908>, <luminance_analysis.FishData object at 0x00000221216EA688>]
[<luminance_analysis.FishData object at 0x0000022120EF8948>, <luminance_analysis.FishData object 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[<luminance_analysis.FishData object at 0x00000221210B8808>, <luminance_analysis.FishData object at 0x00000221210B8888>, <luminance_analysis.FishData object at 0x00000221210BAE08>, <luminance_analysis.FishData object at 0x00000221210B9408>, <luminance_analysis.FishData object at 0x00000221210C59C8>]
[<luminance_analysis.FishData object at 0x00000221210B8E88>, <luminance_analysis.FishData object at 0x00000221210B8DC8>, <luminance_analysis.FishData object at 0x00000221216E3188>, <luminance_analysis.FishData object at 0x00000221210C5708>, <luminance_analysis.FishData object at 0x00000221210BFA48>]
[<luminance_analysis.FishData object at 0x00000221210B73C8>, <luminance_analysis.FishData object at 0x00000221210B72C8>, <luminance_analysis.FishData object at 0x00000221210B9D88>, <luminance_analysis.FishData object at 0x00000221210C5208>, <luminance_analysis.FishData object at 0x000002212139E708>]
[<luminance_analysis.FishData object at 0x00000221216ECC88>, <luminance_analysis.FishData object 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[<luminance_analysis.FishData object at 0x00000221212B9C88>, <luminance_analysis.FishData object at 0x00000221212B9D08>, <luminance_analysis.FishData object at 0x00000221212CAE48>, <luminance_analysis.FishData object at 0x00000221212C8F08>, <luminance_analysis.FishData object at 0x00000221212B2288>]
[<luminance_analysis.FishData object at 0x00000221212B8D08>, <luminance_analysis.FishData object at 0x00000221212B8B88>, <luminance_analysis.FishData object at 0x00000221212B6F08>, <luminance_analysis.FishData object at 0x00000221212C1A08>, <luminance_analysis.FishData object at 0x00000221212C60C8>]
[<luminance_analysis.FishData object at 0x00000221212B7448>, <luminance_analysis.FishData object at 0x00000221212B7408>, <luminance_analysis.FishData object at 0x00000221212B2288>, <luminance_analysis.FishData object at 0x00000221210E3088>, <luminance_analysis.FishData object at 0x00000221212B94C8>]
[<luminance_analysis.FishData object at 0x00000221212CE0C8>, <luminance_analysis.FishData object 

### PC responses panels

In [7]:
from luminance_analysis.plotting import plot_clusters_dendro, re_histogram

In [8]:
color = sns.color_palette()[2]
k = "PC"
spacing = 3
fig_clust = plt.figure(figsize=(7, 6))

#for protocol in ["steps", "flashes"]:

for i, (protocol, dendrolim, cbar, x_pos) in enumerate(zip(["steps", "flashes"], [392, 292], 
                                                           [False, True], [0.45, 0.0])):
    data_dict = protocol_data_dict[protocol]
    stim_ref = PooledData(path = master_path / protocol / "GC").stimarray_rep
    f_hist = re_histogram(data_dict[k]["rel_idxs"], data_dict[k]["rel_thr"], fig_clust,  
                          w=0.18, h=0.08, w_p=0.02, h_p=x_pos+0.42, color=color)

    meanresps = data_dict[k]["resamp_mn_resps"]  # data_dict[k]["deconv_mn_resps"]
    smooth_mean_resps = pd.DataFrame(meanresps.T).rolling(4, center=True).mean().values.T

    fig_clust = plot_clusters_dendro(smooth_mean_resps, stim_ref,
                                 data_dict[k]["linkage_mat"], data_dict[k]["clust_labels"], prefix=k,
                                 figure=fig_clust, w=1., h=0.66, w_p=0.1, h_p=x_pos, f_lim=2,
                                 dendrolims=(dendrolim, 0), gamma=0.4, spacing=spacing, colorbar=cbar)
    
fig_clust.text(0.01, 0.975, 'A')
fig_clust.text(0.01, 0.525, 'B')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[<luminance_analysis.FishData object at 0x00000221212B29C8>, <luminance_analysis.FishData object at 0x00000221212B2A88>, <luminance_analysis.FishData object at 0x00000221210E3108>, <luminance_analysis.FishData object at 0x00000221212B47C8>, <luminance_analysis.FishData object at 0x00000221212D1C48>]


c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:291: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[211 299 341 411 457 543 605 672]


c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.p

[<luminance_analysis.FishData object at 0x00000221212B8308>, <luminance_analysis.FishData object at 0x00000221212D1688>, <luminance_analysis.FishData object at 0x00000221478D1388>, <luminance_analysis.FishData object at 0x0000022121633908>, <luminance_analysis.FishData object at 0x000002212163EE88>]


c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:291: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[103 221 514 690]


c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -


Text(0.01, 0.525, 'B')

In [9]:
if fig_fold is not None:
    fig_clust.savefig(str(fig_fold / "steps_panel.pdf"))

### Supplementary panels with all responses together

In [10]:
fig_fold = Path(r"C:\Users\otprat\Documents\figures\luminance\manuscript_figures\fig6supp")

if not os.path.isdir(fig_fold):
    os.mkdir(fig_fold)

In [11]:
protocol = "steps"
data_dict = protocol_data_dict[protocol]
stim_ref = PooledData(path = master_path / protocol / "GC").stimarray_rep

colors = sns.color_palette()[:3]
fig_clust = plt.figure(figsize=(7, 9))
for i, (k, dendrolim, spacing, cbar, x_pos, color) in enumerate(zip(["GC", "IO", "PC"], [940, 112, 582],
                                                [3, 5, 3], [False, False, True], [0.6, 0.3, 0.0], colors)):
    f_hist = re_histogram(data_dict[k]["rel_idxs"], data_dict[k]["rel_thr"], fig_clust,  
                          w=0.18, h=0.08, w_p=0.02, h_p=x_pos+0.26, color=color)
    
    meanresps = data_dict[k]["resamp_mn_resps"]  # data_dict[k]["deconv_mn_resps"]
    smooth_mean_resps = pd.DataFrame(meanresps.T).rolling(4, center=True).mean().values.T
    
    fig_clust = plot_clusters_dendro(smooth_mean_resps, stim_ref,
                                 data_dict[k]["linkage_mat"], data_dict[k]["clust_labels"], prefix=k,
                                 figure=fig_clust, w=1., h=0.42, w_p=0.1, h_p=x_pos, f_lim=2,
                                 dendrolims=(dendrolim, 0), gamma=0.4, spacing=spacing, colorbar=cbar)

[<luminance_analysis.FishData object at 0x00000221337EEDC8>, <luminance_analysis.FishData object at 0x00000221337EEE48>, <luminance_analysis.FishData object at 0x0000022133800448>, <luminance_analysis.FishData object at 0x000002213380CA08>, <luminance_analysis.FishData object at 0x00000221337FF088>]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:291: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[ 108  328  819  965 1023 1211 1595 2365]


c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.p

[18 32 41 45 54 66]


c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.p

[211 299 341 411 457 543 605 672]


c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.p

In [12]:
if fig_fold is not None:
    fig_clust.savefig(str(fig_fold / "steps_panel.pdf"))

In [13]:
protocol = "flashes"
data_dict = protocol_data_dict[protocol]
stim_ref = PooledData(path = master_path / protocol / "GC").stimarray_rep

colors = sns.color_palette()
fig_clust = plt.figure(figsize=(7, 9))
for i, (k, dendrolim, spacing, cbar, x_pos, color) in enumerate(zip(["GC", "IO", "PC"], [940, 112, 582],
                                                [3, 5, 3], [False, False, True], [0.6, 0.3, 0.0], colors)):
    f_hist = re_histogram(data_dict[k]["rel_idxs"], data_dict[k]["rel_thr"], fig_clust,  
                          w=0.18, h=0.08, w_p=0.02, h_p=x_pos+0.26, color=color)
    
    meanresps = data_dict[k]["resamp_mn_resps"]  # data_dict[k]["deconv_mn_resps"]
    smooth_mean_resps = pd.DataFrame(meanresps.T).rolling(4, center=True).mean().values.T
    
    fig_clust = plot_clusters_dendro(smooth_mean_resps, stim_ref,
                                 data_dict[k]["linkage_mat"], data_dict[k]["clust_labels"], prefix=k, 
                                 figure=fig_clust, w=1., h=0.42, w_p=0.1, h_p=x_pos, f_lim=2,
                                 dendrolims=(dendrolim, 0), gamma=0.4, spacing=spacing, colorbar=cbar)

[<luminance_analysis.FishData object at 0x000002213377CDC8>, <luminance_analysis.FishData object at 0x000002216546F988>, <luminance_analysis.FishData object at 0x0000022165471E48>, <luminance_analysis.FishData object at 0x000002216548A408>, <luminance_analysis.FishData object at 0x0000022165485A88>]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:291: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[ 445  874 1379 2570]


c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:291: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[ 2  8 10 13 26]


c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:291: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[103 221 514 690]


c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -
c:\users\otprat\repos\luminance_analysis\luminance_analysis\plotting.py:229: RuntimeWarning: Mean of empty slice
  np.nanmean(traces[labels == i, :], 0) -


In [14]:
if fig_fold is not None:
    fig_clust.savefig(str(fig_fold / "flashes_panel.pdf"))

#### Fish contributions to each cluster

In [15]:
from luminance_analysis.plotting import add_offset_axes

In [16]:
def plot_fish_contribution(protocol_data_dict, figure=None, frame=None):
    
    if figure is None:
        figure = plt.figure(figsize=(9, 3))
        
    barWidth = 0.85
    n_fish = 5
    colors = (sns.color_palette("deep", 10))
    fish_contribution = {protocol:{} for protocol in protocol_data_dict.keys()}
    brain_region = "PC"
      
    # Figure formatting parameters:
    off = 0.1
    size_f = 0.05
    k_off = off
    
    for i, protocol in enumerate(protocol_data_dict.keys()):
        data_dict = protocol_data_dict[protocol]
        
        clusters = np.unique(data_dict[brain_region]['clust_labels'])
        n_clust = clusters.shape[0]
        roi_map = data_dict[brain_region]['pooled_data'].roi_map

        for fish in range(n_fish):
            fish_labels = data_dict[brain_region]['clust_labels'][roi_map[0, :] == fish]
            fish_contribution[protocol]['{} Fish {}'.format(brain_region, fish+1)] = np.array([np.sum(fish_labels == c) for c in range(n_clust)])

        contributions_df = pd.DataFrame(fish_contribution[protocol])
        ax_hist = add_offset_axes(figure, (k_off, 0.15, size_f*len(contributions_df), .7), frame=frame)
        k_off += size_f*len(contributions_df) + off
        
        
        for i, c in enumerate(contributions_df.columns):
            ax_hist.bar(clusters+1, contributions_df[c], 
                        bottom=sum([contributions_df[prev] for prev in list(contributions_df.columns)[:i]]),
                    width=barWidth, label=c, color=colors[i])

        #ax_hist.legend()
        ax_hist.set_xlabel("Cluster #")
        ax_hist.set_ylabel("Number of ROIs")
        ax_hist.text(.5, 1, protocol.capitalize(), ha='center', va='top', transform=ax_hist.transAxes, fontsize=8.5)
        
    plt.tight_layout()    
    return(figure)

In [17]:
fig_fish_contrib = plot_fish_contribution(protocol_data_dict)

fig_fish_contrib.text(.025, .9, 'A')
fig_fish_contrib.text(.525, .9, 'B')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\tools\miniconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:43: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


Text(0.525, 0.9, 'B')

In [18]:
if fig_fold is not None:
    fig_fish_contrib.savefig(str(fig_fold / "Fish_contributions.pdf"))